<a href="https://colab.research.google.com/github/lozrigby/lab-sql-query-from-table-names/blob/main/lab_sql_query_from_table_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
# from openai import OpenAI
# import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('Key_from_ironhack')

In [ ]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [ ]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': ['Employee information', 'Salary details for each year', 'Educational studies']
}
df = pd.DataFrame(data)
print(df)

       table                    definition
0  employees          Employee information
1     salary  Salary details for each year
2    studies           Educational studies


In [ ]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [ ]:
print(text_tables)

employees: Employee information
salary: Salary details for each year
studies: Educational studies


In [ ]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [ ]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='What is the average salary')

In [ ]:
print(return_OAI(pqt1))

```json
{
    "tables": ["salary"]
}
```


In [ ]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='what subject is the most popular?')

In [ ]:
print(return_OAI(pqt3))

```json
{
    "tables": {
        "studies": "Educational studies"
    }
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

When asking about sql theories they all seem to be justiable at a glance.  Adding a clause that if it doesnt know then state that seems to mitigate hallucinations.